# Performing Variant Calling

When performing variant calling we need the aligned sequences in SAM or BAM format and the reference genome that we want to call varants against.

First, check you are in the correct directory. 

In [ ]:
pwd

It should display something like:

  ``NGScourse_22/variant_calling/data``

## Generating pileup

The command `samtools mpileup` prints the read bases that align to each position in the reference genome. Type the command:


In [ ]:
samtools mpileup -f /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa SRR13882963.sorted.bam | less -S

Each line corresponds to a position on the genome. 

The columns are: chromosome, position, reference base, read depth, read bases (dot . and comma , indicate match on the forward and on the reverse strand; ACGTN and acgtn a mismatch on the forward and the reverse strand) and the final column is the base qualities encoded into characters. The caret symbol ^ marks the start of a read, the dollar sign $ the end of a read, deleted bases are represented by asterisk *.

This output can be used for a simple consensus calling. One rarely needs this type of output. Instead, for a more sophisticated variant calling method, see the next section.

## Generating genotype likelihoods and calling variants

The `bcftools mpileup` command can be used to generate genotype likelihoods. (Beware: the command mpileup is present in both samtools and bcftools, but in both they do different things. While `samtools mpileup` produces the text pileup output seen in the previous exercise, `bcftools mpileup` generates a VCF file with genotype likelihoods.)

Run the following command (when done, press q to quit the viewing mode):

bcftools mpileup -f /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa SRR13882963.sorted.bam -o SRR13882963.pileup


bcftools call -mv -Ov SRR13882963.pileup --ploidy 1 -o SRR13882963_snp.vcf

This generates an intermediate output which contains genotype likelihoods and other raw information necessary for variant calling. This output is usually streamed directly to the caller like this

 `bcftools mpileup -f /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa SRR13882963.sorted.bam - | bcftools call -mv -o --ploidy 1 SRR13882963_snp.vcf`

The output above contains both variant and non-variant positions. Check the input/output options section of the `bcftools call` usage page and see if there is an option to print out only variant sites. Then construct a command to print out variant sites only:

The INFO and FORMAT fields of each entry tells us something about the data at the position in the genome. It consists of a set of key-value pairs with the tags being explained in the header of the VCF file (see the ##INFO and ##FORMAT lines in the header).

We can tell mpileup to add additional ##INFO and ##FORMAT information to the output. For example, we can ask it to add the FORMAT/AD tag which informs about the number of high-quality reads that support alleles listed in REF and ALT columns. The list of all available tags can be printed with the command:

In [ ]:
bcftools mpileup -a ?

Now let's run the variant calling again, this time adding the `-a AD` option. We will also add the `-Ou` option so that it streams a binary uncompressed BCF into call. This is to avoid the unnecessary CPU overhead of formatting the internal binary format to plain text VCF only to be immediately formatted back to the internal binary format again.

In [ ]:
bcftools mpileup -a AD -f /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz SRR12165160.sorted.bam -Ou | bcftools call -mv -o SRR12165160.vcf

Congratulations, you have sucessfully called variants from some NGS data. Now continue to the next section of the tutorial: [Annotation](Annotation.ipynb)